In [1]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

2025-07-09 21:06:55.618765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752095215.811274      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752095215.870174      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# 📁 Rutas base en Kaggle
images_dir = '/kaggle/input/food41/images'  # Ruta donde se encuentran las imágenes
meta_dir = '/kaggle/input/food41/meta/meta'  # Ruta donde se encuentran los archivos meta (train.txt, test.txt)
output_base = '/kaggle/working/food41_split'  # Ruta de salida donde se organizarán los datos

# Crear las carpetas necesarias /train y /test con subcarpetas por clase
for split in ['train', 'test']:
    for class_name in os.listdir(images_dir):
        os.makedirs(os.path.join(output_base, split, class_name), exist_ok=True)

# Leer archivos de división
train_file = os.path.join(meta_dir, 'train.txt')
test_file = os.path.join(meta_dir, 'test.txt')

with open(train_file, 'r') as f:
    train_list = [line.strip() for line in f]

with open(test_file, 'r') as f:
    test_list = [line.strip() for line in f]

# Copiar imágenes a sus carpetas correspondientes para entrenamiento
for item in train_list:
    src = os.path.join(images_dir, item + '.jpg')  # Ruta de la imagen original
    dst = os.path.join(output_base, 'train', item.split('/')[0], item.split('/')[1] + '.jpg')  # Ruta de destino
    if os.path.exists(src):
        shutil.copy(src, dst)

# Copiar imágenes a sus carpetas correspondientes para prueba
for item in test_list:
    src = os.path.join(images_dir, item + '.jpg')  # Ruta de la imagen original
    dst = os.path.join(output_base, 'test', item.split('/')[0], item.split('/')[1] + '.jpg')  # Ruta de destino
    if os.path.exists(src):
        shutil.copy(src, dst)

print("✅ Imágenes reorganizadas correctamente en /kaggle/working/food41_split/train y /test.")


✅ Imágenes reorganizadas correctamente en /kaggle/working/food41_split/train y /test.


In [3]:
# Crear un generador de datos para entrenamiento con aumento de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normaliza las imágenes
    rotation_range=40,  # Aumenta la variabilidad de la orientación de las imágenes
    width_shift_range=0.2,  # Permite mover las imágenes horizontalmente
    height_shift_range=0.2,  # Permite mover las imágenes verticalmente
    shear_range=0.2,  # Rotación arbitraria
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Permite reflejar horizontalmente las imágenes
    fill_mode='nearest'  # Cómo rellenar los píxeles vacíos al realizar transformaciones
)

# Crear un generador de datos para validación (sin aumento de datos)
test_datagen = ImageDataGenerator(rescale=1./255)

# Crear generadores para cargar imágenes desde las carpetas
train_generator = train_datagen.flow_from_directory(
    os.path.join(output_base, 'train'),
    target_size=(299, 299),  # Xception requiere imágenes de 299x299
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(output_base, 'test'),
    target_size=(299, 299),  # Xception requiere imágenes de 299x299
    batch_size=32,
    class_mode='categorical'
)

print("✅ Generadores de datos creados exitosamente.")

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
✅ Generadores de datos creados exitosamente.


In [ ]:
# Cargar el modelo preentrenado Xception
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Congelar las capas del modelo base para evitar que se entrenen
for layer in base_model.layers:
    layer.trainable = False

# Agregar capas adicionales para clasificación
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(101, activation='softmax')(x)  # 101 clases de Food-101

# Crear el modelo completo
model = models.Model(inputs=base_model.input, outputs=x)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

Model: "functional_2"

 Trainable params: 2,201,701 (8.40 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [ ]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=15,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

print("✅ Entrenamiento completado.")

Epoch 1/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 1740s 731ms/step - accuracy: 0.3849 - loss: 2.5285 - val_accuracy: 0.5852 - val_loss: 1.5276
Epoch 2/15
   1/2367 ━━━━━━━━━━━━━━━━━━━━ 9:44 247ms/step - accuracy: 0.5000 - loss: 1.9015

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2367/2367 ━━━━━━━━━━━━━━━━━━━━ 197s 83ms/step - accuracy: 0.5000 - loss: 1.9015 - val_accuracy: 0.5851 - val_loss: 1.5271
Epoch 3/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 1714s 724ms/step - accuracy: 0.5386 - loss: 1.7791 - val_accuracy: 0.6156 - val_loss: 1.4008
Epoch 4/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 196s 83ms/step - accuracy: 0.5000 - loss: 1.6600 - val_accuracy: 0.6166 - val_loss: 1.4004
Epoch 5/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 1709s 722ms/step - accuracy: 0.5715 - loss: 1.6343 - val_accuracy: 0.6265 - val_loss: 1.3726
Epoch 6/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 197s 83ms/step - accuracy: 0.5312 - loss: 1.9539 - val_accuracy: 0.6261 - val_loss: 1.3708
Epoch 7/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 1701s 719ms/step - accuracy: 0.5874 - loss: 1.5543 - val_accuracy: 0.6284 - val_loss: 1.3599
Epoch 8/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 197s 83ms/step - accuracy: 0.6250 - loss: 1.5912 - val_accuracy: 0.6280 - val_loss: 1.3631
Epoch 9/15
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 1713s 724ms/step - accuracy: 0.6

In [10]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

790/790 ━━━━━━━━━━━━━━━━━━━━ 201s 255ms/step - accuracy: 0.6407 - loss: 1.3248
Test Accuracy: 64.17%


In [ ]:
# Graficar precisión de entrenamiento vs. validación
plt.plot(history.history['accuracy'], label='Precisión de entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de validación')
plt.title('Precisión del modelo durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

# Graficar pérdida de entrenamiento vs. validación
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Pérdida del modelo durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

In [11]:
# Guardar el modelo entrenado en formato .h5
model.save('/kaggle/working/xception_food101.h5')
print("✅ El modelo se ha guardado exitosamente como 'food101_xception_model.h5'.")

✅ El modelo se ha guardado exitosamente como 'food101_xception_model.h5'.
